In [15]:
import psycopg2
import secrets
import pandas as pd
import os
import shapely
import datetime

In [16]:
db_name = 'tweets_paul'
user = secrets.user_paul
password = secrets.password_paul
host = '74.207.246.217'

In [17]:
connstring = "dbname={} user={} host={} password={}".format(db_name, user, host, password)

In [24]:
conn = psycopg2.connect(database=db_name, 
                        user=user,
                        password=password,
                        host=host)

In [19]:
cur = conn.cursor()
cur.execute("""SELECT srtext FROM spatial_ref_sys WHERE srid = 32610;""")
rows = cur.fetchall()

In [20]:
# Check spatial settings for postgis database

print rows[0][0]

PROJCS["WGS 84 / UTM zone 10N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-123],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32610"]]


In [7]:
# load data

with pd.HDFStore(os.path.join(os.getcwd(), '..', 'data', 'tweets_1M.h5'), mode='r') as store:
    df = store.tweets_subset

In [8]:
df.head()

,id,lat,lng,text,timeStamp,user_id
765125,384431028661526528,37.895140,-122.101307,Eagles fans are the most bipolar people ever h...,2013-09-29 21:34:34,248989689
308375,380727746852171777,37.933621,-122.336086,My emails tho http://t.co/ctPJWBbGHO,2013-09-19 16:19:03,234263178
181260,379675732739633152,37.785864,-122.403037,Mexican (@ Jessie Square) http://t.co/Dd2dU95exI,2013-09-16 18:38:43,556086698
355895,381122524378968064,37.409624,-121.937065,A hoe will be a hoe &amp; a nigga will be a ni...,2013-09-20 18:27:45,118594397
428001,381658997150851072,37.981324,-122.332549,There' goes my babbbbby,2013-09-22 05:59:30,585032848


In [6]:
# Duplicate SQL connection using sqlalchemy to take advantage of 
# pandas built in "to_sql" method

from sqlalchemy import create_engine
# enginestring = 'postgresql://{}:{}@74.207.246.217:5432/tweets_paul'.format(user, password)
engine = create_engine(enginestring)

In [28]:
connection = engine.connect()
result = connection.execute("SELECT srtext FROM spatial_ref_sys WHERE srid = 32610;")
for row in result:
    print row

NameError: name 'engine' is not defined

In [ ]:
# Load data to SQL

df.to_sql('tweets', engine)